<a href="https://colab.research.google.com/github/OtavioCarvalho1/Time_Series/blob/main/SHM_ML_Vers%C3%A3o_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/OtavioCarvalho1/Time_Series.git

Cloning into 'Time_Series'...
remote: Enumerating objects: 1250, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1250 (delta 2), reused 0 (delta 0), pack-reused 1244
Receiving objects: 100% (1250/1250), 169.35 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Checking out files: 100% (1208/1208), done.


In [ ]:
!pip install statsmodels --upgrade

     |████████████████████████████████| 9.5 MB 29.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from scipy.stats import kurtosis, skew

from sklearn.preprocessing import StandardScaler

from math import sqrt

from statsmodels.tsa.ar_model import AutoReg

from statistics import stdev

# **Importação dos Sinais**

In [ ]:
sem_dano = get_features("/content/Time_Series/Dataset-LambWave-Wagner/PL0402_Ref/PL0402_Ref2_1_150KHz__5ciclos_B3_A3.csv")

dano_1 = get_features("/content/Time_Series/Dataset-LambWave-Wagner/PL0402_Dn1/PL0402_Dn12_1_150KHz__5ciclos_B3_A3.csv")

dano_2 = get_features("/content/Time_Series/Dataset-LambWave-Wagner/PL0402_Dn2/PL0402_Dn22_1_150KHz__5ciclos_B3_A3.csv")

dano_3 = get_features("/content/Time_Series/Dataset-LambWave-Wagner/PL0402_Dn3/PL0402_Dn32_1_150KHz__5ciclos_B3_A3.csv")

dano_4 = get_features("/content/Time_Series/Dataset-LambWave-Wagner/PL0402_Dn4/PL0402_Dn42_1_150KHz__5ciclos_B3_A3.csv")

# **Métricas de dano**

In [ ]:
def get_features(arquivo):
    names = ['Tempo [s]', 'Amplitude PZT Atuador', 'Amplitude PZT Sensor']
    df = pd.read_csv(arquivo, names=names)
    peak = np.max(df['Amplitude PZT Sensor']) #1 - peak
    mean = df['Amplitude PZT Sensor'].mean() #2 - mean
    ms = np.square(mean) #3 - mean square
    rms = np.sqrt(ms) #4 - root mean square
    var = df['Amplitude PZT Sensor'].var() #5 - variance
    std = stdev(df['Amplitude PZT Sensor']) #6 - standard deviation
    skw = skew(df['Amplitude PZT Sensor']) #7 - skewness
    kurt = kurtosis(df['Amplitude PZT Sensor']) #8 - kurtosis
    crest_f = peak/rms #9 - crest factor
    K_f = peak*rms #10 - K-Factor
    md = pd.DataFrame([peak, mean, ms, rms, var, std, skw, kurt, crest_f, K_f])
    return md

In [ ]:
frames = [sem_dano,dano_1,dano_2,dano_3,dano_4]

features = pd.concat(frames, axis=1)

features

,0,0,0,0,0
0,1.123057e-02,1.199896e-02,1.286964e-02,1.391785e-02,1.517050e-02
1,-1.152493e-07,-4.716260e-07,-2.854587e-07,-2.894220e-07,-1.540067e-08
2,1.328241e-14,2.224311e-13,8.148665e-14,8.376509e-14,2.371805e-16
3,1.152493e-07,4.716260e-07,2.854587e-07,2.894220e-07,1.540067e-08
4,3.207258e-06,3.592699e-06,3.648665e-06,3.927971e-06,4.227067e-06
5,1.790882e-03,1.895442e-03,1.910148e-03,1.981911e-03,2.055983e-03
6,7.012031e-04,9.708261e-04,5.890596e-04,1.178199e-03,8.088033e-04
7,9.654932e+00,1.042437e+01,1.169644e+01,1.155114e+01,1.181413e+01
8,9.744586e+04,2.544168e+04,4.508408e+04,4.808843e+04,9.850548e+05
9,1.294316e-09,5.659022e-09,3.673750e-09,4.028132e-09,2.336358e-10


In [ ]:
#1 peak amplitude
peak = np.max(df_train['Amplitude PZT Sensor'])

peak

In [ ]:
#2 mean
mean = df_train['Amplitude PZT Sensor'].mean()

mean

In [ ]:
#3 mean square
mean_square = np.square(mean)

mean_square

In [ ]:
#4 root mean square
rms = np.sqrt(mean_square)

rms

In [ ]:
#5 variance
df_train['Amplitude PZT Sensor'].var()

In [ ]:
#6 standard deviation
stdev(df_train['Amplitude PZT Sensor'])

In [ ]:
#7 skewness
skew(df_train['Amplitude PZT Sensor'])

In [ ]:
#8 kurtosis
kurtosis(df_train['Amplitude PZT Sensor'])

In [ ]:
#9 crest factor
peak/rms

In [ ]:
#10 K-factor
peak*rms